Articles for training  - Emails (ground truth)
=========================


****

In [402]:
import os
import re
import json
import pprint
import datetime
import pandas as pd

pp = pprint.PrettyPrinter(indent=4)

In [403]:
data = []
with open('../articles_email.json') as input_file:
    for line in input_file:
        data.append(json.loads(line))

In [404]:
df = pd.DataFrame(data)

print("Number of articles downloaded: {:,}".format(df.shape[0]))

Number of articles downloaded: 4,209


In [421]:
print("Min date:", df[df['publish_date'].notna()]['publish_date'].min()[:10])
print("Max date:", df[df['publish_date'].notna()]['publish_date'].max()[:10])

Min date: 2016-02-04
Max date: 2018-04-16


### Duplicates and useless articles

In [389]:
df_dups = df.groupby('text').size().sort_values(0,ascending=False).reset_index().rename(columns={0:"count"})
df_dups.head(10)

,text,count
0,,70
1,How Can We Help?\n\nIf you'd like to learn mor...,35
2,Estas personas se consideran víctimas y no cóm...,5
3,Don't have an account yet?\n\nThe content of t...,4
4,Sucesión de mentirijillas en BBVA: Francisco G...,4
5,Consulte las citas más relevantes de la jornad...,4
6,El ministro de Economía aseguró que la resoluc...,3
7,"Danièle Nouy, presidenta del Supervisor Único,...",3
8,El Royal Bank of Scotland (RBS) pagará 500 mil...,2
9,La tecnología blockchain se está convirtiendo ...,2


In [390]:
# text 1: How Can We Help?\n\nIf you'd like to learn mor...
print("Text 1:\n")
print(df_dups.loc[1]['text'],"...")

idx = df.index[df['text']==df_dups.loc[1]['text']]
df = df.drop(idx)

print("\n\nCurrent number of articles: {:,}".format(df.shape[0]))

Text 1:

How Can We Help?

If you'd like to learn more about Bernstein's insights and execution or how they can help advance your business, please contact us. ...


Current number of articles: 4,174


In [391]:
# text 3: Don't have an account yet?\n\nThe content of t...
idx = df.index[df['text']==df_dups.loc[3]['text']]
df = df.drop(idx)

print("Current number of articles: {:,}".format(df.shape[0]))

Current number of articles: 4,170


In [392]:
# removing empty text articles
df = df.drop(df[df['text']==''].index)
print("Current number of articles: {:,}".format(df.shape[0]))

Current number of articles: 4,100


In [393]:
# removing duplicates
df = df.drop_duplicates(subset='text', keep='first', inplace=False)
print("Current number of articles: {:,}".format(df.shape[0]))

Current number of articles: 4,024


In [394]:
# articles with very short text (useless)
for idx, row in df[df['text'].str.len()<150].iterrows():
    print(idx, "\t", repr(row['text']), "\n")

5 	 'Please enable JavaScript if it is disabled in your browser or access the information through the links provided below.' 

75 	 'Consulte las citas más relevantes de la jornada actual y de las próximas. [Acceder]' 

334 	 'No ha sido posible completar su consulta. Vuelva a intentarlo más tarde.' 

343 	 'Estas personas se consideran víctimas y no cómplices de una situación en la que se encuentran más de 200.000 trabajadores.' 

1836 	 'Agencia Estatal Boletín Oficial del Estado\n\nAvda. de Manoteras, 54 - 28050 Madrid - Tel .: (+34) 91 111 4000' 

2047 	 '¿No estás registrado? Crea tu cuenta\n\nEntra en EL PAÍS\n\nEntra en EL PAÍS\n\nCerrar menú\n\nTamaño letra a- A+\n\nAlto contraste' 

3110 	 'Sucesión de mentirijillas en BBVA: Francisco González pasa el testigo a FG' 



### Creating new features

* section
* datepart
* day of week
* newspaper

In [395]:
# newspaper

df['newspaper'] = df['url'].apply(lambda x: re.findall('(?:\/\/www\.|\/\/)(\w+)\.\w',x))

for idx, row in df.iterrows():
    source = row['newspaper']
    try:
        df.loc[idx, 'newspaper'] = source[0]
    except:
        df.loc[idx, 'newspaper'] = ""

In [396]:
df_source = df.groupby('newspaper').size().sort_values(0, ascending=False).reset_index().rename(columns={0:"count"})
df_source.head(10)

,newspaper,count
0,expansion,2142
1,cincodias,1077
2,elconfidencial,709
3,blogs,29
4,retina,12
5,federalreserve,9
6,economia,6
7,euribor,3
8,bloomberg,3
9,criteria,2


In [398]:
# split the publish date to date and time and remove time because it is not changing
df['date'], df['time'] = df.publish_date.str.split(' ').str
df.drop(['time'], axis=1, inplace=True)

In [399]:
# convert date columns to datetime and create day_of_week column
df['date'] = pd.to_datetime(df['date'])
df['day_of_week'] = df['date'].dt.weekday_name
df.head(5)

,authors,keywords,publish_date,summary,text,title,top_image,url,newspaper,date,day_of_week
3,[],"[se, por, la, el, y, los, las, nota, sobre, de...",2016-03-01 21:27:34+00:00,"Primero la situación económica de China, despu...",Desde que empezó el año hemos tenido ya 3 fact...,Nota sobre la situación de los bancos,http://www.robust.fondos.gvcgaesco.es/wp-conte...,http://www.robustglobal.com/nota-sobre-la-situ...,robustglobal,2016-03-01,Tuesday
4,[],"[statements, minutes, meetings, calendars, foi...",None,"Meeting calendars, statements, and minutes (20...","Meeting calendars, statements, and minutes (20...",Meeting calendars and information,,http://www.federalreserve.gov/monetarypolicy/f...,federalreserve,NaT,NaN
5,[],"[access, provided, javascript, fed, informatio...",None,Please enable JavaScript if it is disabled in ...,Please enable JavaScript if it is disabled in ...,The Fed,,http://www.federalreserve.gov/mediacenter/medi...,federalreserve,NaT,NaN
6,[],"[federal, labor, reserve, funds, fomc, market,...",None,"A range of recent indicators, including strong...",Information received since the Federal Open Ma...,Federal Reserve issues FOMC statement,,https://www.federalreserve.gov/newsevents/pres...,federalreserve,NaT,NaN
7,"[Asunción Infante Fuentes, Andrés Stumpf Guirao]","[vuelve, se, estas, la, son, y, tumbar, los, l...",2016-03-17 00:00:00,"El Ibex cede un 1,18% y pierde el soporte clav...",Amaneció con subidas en todos los grandes parq...,El sector bancario vuelve a tumbar al Ibex: es...,https://d500.epimg.net/cincodias/iconos/v2.x/v...,http://cincodias.com/cincodias/2016/03/17/merc...,cincodias,2016-03-17,Thursday


In [400]:
# drop unnecessary columns
df.drop(['top_image', 'publish_date'], axis=1, inplace=True)

In [401]:
df.columns

Index(['authors', 'keywords', 'summary', 'text', 'title', 'url', 'newspaper',
       'date', 'day_of_week'],
      dtype='object')

---

# Articles for training - Historical data

Checking the newspaper output for all urls scraped from the different source to build the training dataset. The scraped newspapers are (run every 8 days between 2016/04/01 and 2018/04/01):

1. Expansion
2. Cincodias
3. El Confidencial

__Considerations__

* Articles with __no text__.
* Note that some of these __articles may be included in the emails__ sent over these years.
* Currenty working with articles __published on mondays__ regardless of the source.
* Handle errors/exceptions in the download method of the newspaper.

---

## 1. Expansion

In [303]:
# article content downloaded and parsed with newspaper3k
data = []
with open('../articles_expansion_hemeroteca.json') as input_file:
    for line in input_file:
        data.append(json.loads(line))

In [304]:
df = pd.DataFrame(data)

In [305]:
print("Number of articles downloaded: {:,}".format(df.shape[0]))

Number of articles downloaded: 9,986


### Duplicates and useless articles

In [306]:
df_dups = df.groupby('text').size().sort_values(0,ascending=False).reset_index().rename(columns={0:"count"})
df_dups.head(5)

,text,count
0,"Para comentar o valorar, por favor inicie sesi...",87
1,"Para comentar o valorar, por favor inicie sesi...",58
2,,46
3,¿Televisión de pago en España a precios low co...,2
4,"En plena era tecnológica, donde podemos relaci...",2


We can see that there are articles with useless text and articles with no text at all. We will get rid of all these invalid articles.

In [307]:
# text 0: Para comentar o valorar, por favor inicie sesi...
idx = df.index[df['text']==df_dups.loc[0]['text']]
df = df.drop(idx)

# text 1: Para comentar o valorar, por favor inicie sesi...
idx = df.index[df['text']==df_dups.loc[1]['text']]
df = df.drop(idx)

# text 2: (empty text)
idx = df.index[df['text']==df_dups.loc[2]['text']]
df = df.drop(idx)

print("Current number of articles: {:,}".format(df.shape[0]))

Current number of articles: 9,795


In [308]:
df_dups = df.groupby('text').size().sort_values(0,ascending=False).reset_index().rename(columns={0:"count"})
print("Number of articles duplicated twice: {:,}".format(df_dups[df_dups['count']==2]['text'].count()))

Number of articles duplicated twice: 23


In [309]:
# removing duplicates
df = df.drop_duplicates(subset='text', keep='first', inplace=False)

In [310]:
print("Current number of articles: {:,}".format(df.shape[0]))

Current number of articles: 9,772


In [311]:
df_dups = df.groupby('text').size().sort_values(0,ascending=False).reset_index().rename(columns={0:"count"})
df_dups.head(5)

,text,count
0,Único coche del mundo con tres trenes de poten...,1
1,El plan del Gobierno británico permitirá al se...,1
2,"El plazo de amortización es inferior a un año,...",1
3,El plazo de aceptación se extenderá desde maña...,1
4,El plazo de aceptación de la opa que Carlos Sl...,1


There are no more text duplicates at this point. However, there are useless articles such as ...

In [312]:
# articles with very short text (useless)
for idx, row in df[df['text'].str.len()<150].iterrows():
    print(idx, "\t", repr(row['text']), "\n")

904 	 'Este fin de semana con EXPANSIÓN, conozca los fondos más recomendados por los expertos.' 

1166 	 'El presidente del Gobierno en funciones y candidato a la reelección por el Partido Popular, Mariano RajoyJuanJo MartínEFE' 

1722 	 'La tregua alcanza a las bolsas europeas, y a sus divisas. El Ibex sale de mínimos de tres años con el rebote de la banca y de IAG.' 

2464 	 'La comisaria de Competencia de la UE, Margrethe Vestager.Foto: Efe' 

2701 	 '¡Empieza el partido! ¿Quién de estos pilotos de la Scuderia Ferrari será tan bueno jugando voleibol como lo es en la pista?\n\nRemitido' 

3543 	 'El fabricante ha presentado en el Salón de Pekín de China una solución de movilidad para los grandes atascos que combina el coche y el patinete.' 

3677 	 'Francisco R. Checa y Susana Pérez analizan las claves bursátiles del día y adelantan las previsiones del viernes.' 

5106 	 'Esta encuesta está cerrada y no se puede votar en ella.' 

5479 	 'Galería Tubo Solar VELUX\n\nDescubra los benef

### Creating new features

* section
* datepart
* day of week
* newspaper

In [313]:
# newspaper
df['newspaper'] = 'expansion'

In [314]:
# create seccion field from url
df['section'] = df['url'].apply(lambda x: x.split('/')[3])

In [315]:
df.groupby('section').size().sort_values(0, ascending=False).reset_index().rename(columns={0:'count'}).head(10)

,section,count
0,empresas,2188
1,economia,1970
2,mercados,945
3,aragon,777
4,extremadura,604
5,directivos,462
6,economia-digital,380
7,fueradeserie,375
8,sociedad,321
9,juridico,262


In [316]:
# split the publish date to date and time and remove time because it is not changing
df['date'], df['time'] = df.publish_date.str.split(' ').str
df.drop(['time'], axis=1, inplace=True)

In [317]:
# convert date columns to datetime and create day_of_week column
df['date'] = pd.to_datetime(df['date'])
df['day_of_week'] = df['date'].dt.weekday_name
df.head(5)

,authors,keywords,publish_date,summary,text,title,top_image,url,newspaper,section,date,day_of_week
0,[],"[las, terremoto, se, aumenta, que, ha, el, ecu...",2016-04-17 00:00:00,"El presidente de Ecuador, Rafael Correa, ha el...","El presidente de Ecuador, Rafael Correa, ha el...",Aumenta a 272 el número de muertos a causa del...,http://estaticos.expansion.com/assets/desktop/...,http://www.expansion.com/sociedad/2016/04/17/5...,expansion,sociedad,2016-04-17,Sunday
1,[],"[empresa, sauces, presunta, investigan, tambié...",2016-05-03 00:00:00,La Justicia argentina ha iniciado una causa po...,La Justicia argentina ha iniciado una causa po...,Investigan una empresa de Cristina Fernández d...,http://estaticos.expansion.com/assets/multimed...,http://www.expansion.com/economia/politica/201...,expansion,economia,2016-05-03,Tuesday
2,[],"[superiores, las, pedroche, ser, josé, ni, que...",2016-04-17 00:00:00,Cristina Pedroche es de izquierdas y de Vallec...,Cristina Pedroche es de izquierdas y de Vallec...,Ni Cristina Pedroche ni José Sacristán son sup...,http://estaticos.expansion.com/assets/multimed...,http://www.expansion.com/actualidadeconomica/a...,expansion,actualidadeconomica,2016-04-17,Sunday
3,[],"[su, próximo, resultado, que, el, en, del, gre...",2016-04-17 00:00:00,"""El equipo del Fondo volverá a Atenas a princi...",La directora gerente del Fondo Monetario Inter...,"El FMI no espera obtener un ""resultado inmedia...",http://estaticos.expansion.com/assets/multimed...,http://www.expansion.com/economia/2016/04/17/5...,expansion,economia,2016-04-17,Sunday
4,[],"[su, aunque, que, encontrará, investigarme, po...",2016-04-17 00:00:00,"En una entrevista en el diario El País, Soria ...",El ex ministro reconoce que a pesar de que su ...,"Soria: ""Montoro hará bien en investigarme, aun...",http://estaticos.expansion.com/assets/multimed...,http://www.expansion.com/economia/politica/201...,expansion,economia,2016-04-17,Sunday


In [318]:
# drop unnecessary columns
df.drop(['top_image', 'publish_date'], axis=1, inplace=True)

### Dump to JSON

In [319]:
df.to_json('./articles_clean_expansion.json')

In [320]:
pp.pprint(dict(df.loc[0]))

{   'authors': [],
    'date': Timestamp('2016-04-17 00:00:00'),
    'day_of_week': 'Sunday',
    'keywords': [   'las',
                    'terremoto',
                    'se',
                    'aumenta',
                    'que',
                    'ha',
                    'el',
                    'ecuador',
                    'en',
                    'del',
                    '272',
                    'muertos',
                    'causa',
                    'número',
                    'y',
                    'los',
                    'la'],
    'newspaper': 'expansion',
    'section': 'sociedad',
    'summary': 'El presidente de Ecuador, Rafael Correa, ha elevado este '
               'domingo a 272 el número de muertos a causa del terremoto de '
               'magnitud 7,8 en la escala de Richter que sacudió el sábado el '
               'país, agregando que el balance de víctimas "aumentará en las '
               'próximas horas".\n'
               '"Son mome

---

## 2. Cincodias

In [321]:
# article content downloaded and parsed with newspaper3k
data = []        
with open('../articles_cincodias.json') as input_file:
    for line in input_file:
        data.append(json.loads(line))

In [322]:
df = pd.DataFrame(data)

In [323]:
print("Number of articles downloaded: {:,}".format(df.shape[0]))

Number of articles downloaded: 3,998


### Duplicates and useless articles

In [324]:
df_dups = df.groupby('text').size().sort_values(0,ascending=False).reset_index().rename(columns={0:"count"})
df_dups.head(5)

,text,count
0,¿No estás registrado? Crea tu cuenta\n\nEntra ...,26
1,Europa ha empezado ya con un particular carrus...,2
2,Hablemos de democracia. Si quieren hablemos en...,2
3,Luis de Guindos está a poco más de un paso de ...,2
4,"El euríbor a 12 meses, el indicador más utiliz...",2


In [325]:
# text 0: ¿No estás registrado? Crea tu cuenta\n\nEntra ...
idx = df.index[df['text']==df_dups.loc[0]['text']]
df = df.drop(idx)

print("Current number of articles: {:,}".format(df.shape[0]))

Current number of articles: 3,972


In [326]:
df_dups = df.groupby('text').size().sort_values(0,ascending=False).reset_index().rename(columns={0:"count"})
print("Number of articles duplicated twice: {:,}".format(df_dups[df_dups['count']==2]['text'].count()))

Number of articles duplicated twice: 6


In [327]:
# removing duplicates
df = df.drop_duplicates(subset='text', keep='first', inplace=False)
print("Current number of articles: {:,}".format(df.shape[0]))

Current number of articles: 3,966


In [328]:
df_dups = df.groupby('text').size().sort_values(0,ascending=False).reset_index().rename(columns={0:"count"})
df_dups.head(5)

,text,count
0,“¡Que no le falte de nada a mis #materialadict...,1
1,"El principal indicador de la bolsa española, e...",1
2,"El principal indicador de la Bolsa española, e...",1
3,El primer trimestre del año deja en los invers...,1
4,El primer trimestre de año ha estado repleto d...,1


In [329]:
# removing empty text articles
df = df.drop(df[df['text']==''].index)
print("Current number of articles: {:,}".format(df.shape[0]))

Current number of articles: 3,965


In [330]:
# articles with very short text (useless)
for idx, row in df[df['text'].str.len()<150].iterrows():
    print(idx, "\t", repr(row['text']), "\n")

57 	 'El coste de los despidos en el sector financiero\n\nCompatir en Linkedin Enviar por correo Subir' 

79 	 'Los problemas a los que se enfrentan los bancos europeos\n\nCompatir en Linkedin Enviar por correo Subir' 

243 	 'Los creadores de la serie de HBO "Silicon Valley" creen que la tecnología está plagada de "hippies"\n\nCompatir en Linkedin Enviar por correo Subir' 

459 	 'José Padilla, Padilla Supermercados-Spar: “El éxito de nuestro negocio es estar a la altura de las necesidades de nuestro cliente”.' 

480 	 'Si apuestas en la Eurocopa 2016, estas webs predicen el resultado de cada partido\n\nCompatir en Linkedin Enviar por correo Subir' 

632 	 'Cuatro empresas se jugaban las adjudicaciones de las obras con el número del Gordo de Navidad.' 

806 	 '3\n\nLa selección de EE UU de baloncesto, uno de los grandes atractivos de estos juegos, arrolló a China (119-62) en su primer partido en Rio.' 

915 	 'Sánchez da un portazo a Rajoy con un "no rotundo"\n\nAtlas' 

993 	 'Hay un

### Creating new features

* section
* datepart
* day of week
* newspaper

In [331]:
# newspaper
df['newspaper'] = 'cincodias'

In [332]:
# create seccion field from url
df['section'] = df['url'].apply(lambda x: x.split('/')[7])

In [333]:
df.groupby('section').size().sort_values(0, ascending=False).reset_index().rename(columns={0:'count'}).head(10)

,section,count
0,mercados,792
1,companias,686
2,empresas,618
3,economia,464
4,lifestyle,197
5,midinero,196
6,sentidos,135
7,smartphones,129
8,fortunas,105
9,autonomos,99


In [334]:
# split the publish date to date and time and remove time because it is not changing
df['date'], df['time'] = df.publish_date.str.split(' ').str
df.drop(['time'], axis=1, inplace=True)

In [335]:
# convert date columns to datetime and create day_of_week column
df['date'] = pd.to_datetime(df['date'])
df['day_of_week'] = df['date'].dt.weekday_name
df.head(5)

,authors,keywords,publish_date,summary,text,title,top_image,url,newspaper,section,date,day_of_week
0,[Carlos González Villamil],"[y, videoconsola, en, uso, la, mac, que, para,...",2016-04-09 00:00:00,El firmware 3.50 de PlayStation 4 ha mejorado ...,El firmware 3.50 de PlayStation 4 ha mejorado ...,Cómo configurar Remote Play en PC y Mac para e...,https://d500.epimg.net/cincodias/imagenes/2016...,https://cincodias.elpais.com/cincodias/2016/04...,cincodias,lifestyle,2016-04-09,Saturday
1,[Juande Portillo],"[y, los, la, en, que, reduciendo, adelgaza, ba...",2016-04-08 00:00:00,"Una dieta a largo plazo, en todo caso, a la qu...","En apenas una década, y con una crisis financi...",La banca adelgaza costes reduciendo red y plan...,https://d500.epimg.net/cincodias/imagenes/2016...,https://cincodias.elpais.com/cincodias/2016/04...,cincodias,mercados,2016-04-08,Friday
2,[],"[y, los, en, la, han, que, trabajadores, prote...",2016-04-09 00:00:00,"Los trabajadores de Lauki, cuya factoría de Va...","Los trabajadores de Lauki, cuya factoría de Va...",Protesta de la plantilla de Lauki por el cierr...,https://d500.epimg.net/cincodias/imagenes/2016...,https://cincodias.elpais.com/cincodias/2016/04...,cincodias,empresas,2016-04-09,Saturday
3,"[Javier Fernández Magariño, Pablo Monge]","[será, nacerá, y, los, la, en, que, europa, ai...",2016-04-09 00:00:00,Air Europa Express nace sobre la pequeña aerol...,"Juan José Hidalgo, presidente de Globalia y Ai...",Esta será la nueva Air Europa 'low cost' que n...,https://d500.epimg.net/cincodias/imagenes/2016...,https://cincodias.elpais.com/cincodias/2016/04...,cincodias,empresas,2016-04-09,Saturday
4,[Will Oliver],"[y, último, su, en, la, año, que, cameron, el,...",2016-04-09 00:00:00,"WILL OLIVER EFEDavid Cameron, primer ministro ...",Protestas en Londres tras conocer la conexión ...,Cameron pagó impuestos por 90.000 euros en el ...,https://d500.epimg.net/cincodias/imagenes/2016...,https://cincodias.elpais.com/cincodias/2016/04...,cincodias,economia,2016-04-09,Saturday


In [336]:
# drop unnecessary columns
df.drop(['top_image', 'publish_date'], axis=1, inplace=True)

### Dump to JSON

In [337]:
df.to_json('./articles_clean_cincodias.json')

In [338]:
pp.pprint(dict(df.loc[1]))

{   'authors': ['Juande Portillo'],
    'date': Timestamp('2016-04-08 00:00:00'),
    'day_of_week': 'Friday',
    'keywords': [   'y',
                    'los',
                    'la',
                    'en',
                    'que',
                    'reduciendo',
                    'adelgaza',
                    'banca',
                    'el',
                    'red',
                    'costes',
                    'por',
                    'es',
                    'plantilla',
                    'del'],
    'newspaper': 'cincodias',
    'section': 'mercados',
    'summary': 'Una dieta a largo plazo, en todo caso, a la que precederá de '
               'forma inmediata una nueva purga detonada por la presión que '
               'ejercen sobre los márgenes los tipos cero y la debilidad '
               'general del negocio.\n'
               'Un ajuste adicional que rondaría el 20% de la red y la '
               'plantilla actual.\n'
               'Y eso, que 

---

## El Confidencial

In [339]:
# article content downloaded and parsed with newspaper3k
data = []
with open('../articles_elconfidencial.json') as input_file:
    for line in input_file:
        data.append(json.loads(line))

In [340]:
df = pd.DataFrame(data)

In [341]:
print("Number of articles downloaded: {:,}".format(df.shape[0]))

Number of articles downloaded: 5,489


### Duplicates and useless articles

In [342]:
df_dups = df.groupby('text').size().sort_values(0,ascending=False).reset_index().rename(columns={0:"count"})
df_dups.head(5)

,text,count
0,"BIOGRAFÍA\n\nTrader, empezó su carrera profesi...",36
1,,13
2,Estimado lector: el próximo 1 de Junio la Tabl...,8
3,Gonzalo de Diego Ramos\n\nLa mítica empresa de...,4
4,"Entrevista a Attila Szabo\n\n""Un 3% de los run...",3


In [343]:
# text 0: BIOGRAFÍA\n\nTrader, empezó su carrera profesi...
idx = df.index[df['text']==df_dups.loc[0]['text']]
df = df.drop(idx)

print("Current number of articles: {:,}".format(df.shape[0]))

Current number of articles: 5,453


In [344]:
# removing empty text articles
df = df.drop(df[df['text']==''].index)
print("Current number of articles: {:,}".format(df.shape[0]))

Current number of articles: 5,440


In [345]:
# text 2: Estimado lector: el próximo 1 de Junio la Tabl...
print("Text 2:\n")
print(df_dups.loc[2]['text'][:500],"...")

idx = df.index[df['text']==df_dups.loc[2]['text']]
df = df.drop(idx)

print("\n\nCurrent number of articles: {:,}".format(df.shape[0]))

Text 2:

Estimado lector: el próximo 1 de Junio la Tabla de Seguimiento dejará de publicarse en abierto. Por favor suscríbase gratuitamente en la web de Ágora EAFI para visualizarla.

Con el fin de poder comprobar la evolución de las estrategias abiertas por el analista técnico de El Confidencial, Carlos Doblado, se ofrecerá y se actualizará a diario esta tabla de Seguimiento de las operaciones.

La Tabla de Seguimiento se modifica diariamente. Se indica con color rojo aquellas partes de la misma que hay ...


Current number of articles: 5,432


In [346]:
# text 3: Gonzalo de Diego Ramos\n\nLa mítica empresa de...
print("Text 3:\n")
print(df_dups.loc[3]['text'])

idx = df.index[df['text']==df_dups.loc[3]['text']]
df = df.drop(idx)

print("\n\nCurrent number of articles: {:,}".format(df.shape[0]))

Text 3:

Gonzalo de Diego Ramos

La mítica empresa de sándwiches ha dado un giro radical. Sus clientes han cambiado y lo mismo han hecho los productos de su carta y los números de sus balances. ¿Cómo se explica su transformación?


Current number of articles: 5,428


In [347]:
# text 4: Entrevista a Attila Szabo\n\n"Un 3% de los run...
print("Text 4:\n")
print(df_dups.loc[4]['text'])

idx = df.index[df['text']==df_dups.loc[4]['text']]
df = df.drop(idx)

print("\n\nCurrent number of articles: {:,}".format(df.shape[0]))

Text 4:

Entrevista a Attila Szabo

"Un 3% de los runners son adictos, del mismo modo que lo es un adicto al tabaco o a la heroína"


Current number of articles: 5,425


In [348]:
df_dups = df.groupby('text').size().sort_values(0,ascending=False).reset_index().rename(columns={0:"count"})
df_dups.head(5)

,text,count
0,"Coches que conducen solos, robots que cocinan ...",2
1,"“#PasoteaPasote”, describía Fernando Alonso en...",2
2,La Comunidad Laboral Trabajando.com - Universi...,2
3,La profundidad mínima de los neumáticos es de ...,2
4,El viernes pasado día 5 por la tarde fui a lle...,2


In [349]:
# removing duplicates
df = df.drop_duplicates(subset='text', keep='first', inplace=False)
print("Current number of articles: {:,}".format(df.shape[0]))

Current number of articles: 5,291


In [350]:
# articles with very short text (useless)
for idx, row in df[df['text'].str.len()<150].iterrows():
    print(idx, "\t", repr(row['text']), "\n")

729 	 'Un póster del exsecretario general del PSOE, Pedro Sánchez. (Reuters)' 

1004 	 'Pablo Alborán, Ana Rosa Quintana, Mario Vaquerizo... Todos los invitados a los premios del Museo Chicote\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPantalla completa' 

1096 	 'Eduardo Segovia analiza en su videoblog, Money Talks, algunas de las consecuencias que la victoria de Donald Trump supone para los mercados.' 

1405 	 '"si fuera presidente clinton estaría en la cárcel"\n\nEl debate entre Trump y Clinton, en directo: el magnate sale vivo tras cargar con todo' 

1956 	 'Motociclismo\n\nHistorias del paddock\n\npor Juan Pedro de la Torre' 

2279 	 '2 de\n\nRodajas de limón' 

2589 	 'El tenista serbio ha protagonizado un divertido anuncio para la marca de raquetas Head.' 

2611 	 'Así fue el estreno internacional de la Primera Dama\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPantalla completa' 

2635 	 'Increíble lo sucedido en el Tour de los Alpes. En la bajada de un puerto Stef

### Creating new features

* section
* datepart
* day of week
* newspaper

In [351]:
# newspaper
df['newspaper'] = 'elconfidencial'

In [352]:
# create seccion field from url
df['section'] = df['url'].apply(lambda x: x.split('/')[3])

In [353]:
df.groupby('section').size().sort_values(0, ascending=False).reset_index().rename(columns={0:'count'}).head(10)

,section,count
0,espana,1046
1,multimedia,740
2,deportes,543
3,alma-corazon-vida,384
4,cultura,307
5,empresas,293
6,tecnologia,279
7,mundo,262
8,noticias,210
9,mercados,202


In [354]:
# split the publish date to date and time and remove time because it is not changing
df['date'], df['time'] = df.publish_date.str.split(' ').str
df.drop(['time'], axis=1, inplace=True)

In [355]:
# convert date columns to datetime and create day_of_week column
df['date'] = pd.to_datetime(df['date'])
df['day_of_week'] = df['date'].dt.weekday_name
df.head(5)

,authors,keywords,publish_date,summary,text,title,top_image,url,newspaper,section,date,day_of_week
0,"[José Antonio Navas, Agustín Marco, El Confide...","[autopistas, quiebra, en, el, quitas, que, las...",2016-04-25 00:00:00,La gran banca identificada en torno al G-6 que...,La gran banca identificada en torno al G-6 que...,Autopistas de peaje: La banca huye de las auto...,https://www.ecestaticos.com/imagestatic/clippi...,https://www.elconfidencial.com/empresas/2016-0...,elconfidencial,empresas,2016-04-25,Monday
1,"[José María Olmo, Ángel Martínez, Roberto R. B...","[interior, proteger, terrorismo, en, promoción...",2016-04-01 00:00:00,"Además, los expertos en terrorismo yihadista y...",El Gobierno anunció hace dos semanas que en la...,Terrorismo islamista: Interior asignará 200 gu...,https://www.ecestaticos.com/imagestatic/clippi...,https://www.elconfidencial.com/espana/2016-04-...,elconfidencial,espana,2016-04-01,Friday
2,"[Marcos Lamelas. Barcelona, Contacta Al Autor]","[haya, y, ha, en, el, vida, nuevo, que, activo...",2016-04-12 00:00:00,Esta prioridad no es baladí cuando una de sus ...,Abertis está centrada en alargar la vida de su...,Noticias de Abertis: Abertis espera que haya n...,https://www.ecestaticos.com/imagestatic/clippi...,https://www.elconfidencial.com/empresas/2016-0...,elconfidencial,empresas,2016-04-12,Tuesday
3,"[José Antonio Navas, Gonzalo Jiménez-Blanco, C...","[vínculos, el, en, se, que, árbitro, noticias,...",2016-01-20 00:00:00,Abertis y Bankia emprendieron en 2013 un arbit...,La batalla de las autopistas radiales de Madri...,Autopistas de peaje: El laudo de las radiales ...,https://www.ecestaticos.com/imagestatic/clippi...,https://www.elconfidencial.com/empresas/2016-0...,elconfidencial,empresas,2016-01-20,Wednesday
4,"[R. Ugalde, Contacta Al Autor]","[autopistas, primer, su, en, el, descenso, se,...",2015-12-21 00:00:00,El precio medio de los peajes de la red de aut...,El precio medio de los peajes de la red de aut...,Autopistas de peaje: El peaje de las autopista...,https://www.ecestaticos.com/imagestatic/clippi...,https://www.elconfidencial.com/economia/2015-1...,elconfidencial,economia,2015-12-21,Monday


In [356]:
# drop unnecessary columns
df.drop(['top_image', 'publish_date'], axis=1, inplace=True)

### Dump to JSON

In [357]:
df.to_json('./articles_clean_elconfidencial.json')

In [358]:
pp.pprint(dict(df.loc[1]))

{   'authors': [   'José María Olmo',
                   'Ángel Martínez',
                   'Roberto R. Ballesteros',
                   'Sara De Diego',
                   'Contacta Al Autor'],
    'date': Timestamp('2016-04-01 00:00:00'),
    'day_of_week': 'Friday',
    'keywords': [   'interior',
                    'proteger',
                    'terrorismo',
                    'en',
                    'promoción',
                    'el',
                    'islamista',
                    'que',
                    'las',
                    'civiles',
                    'nueva',
                    'guardias',
                    'del',
                    'nucleares',
                    'la',
                    'por',
                    'los',
                    'y',
                    'para'],
    'newspaper': 'elconfidencial',
    'section': 'espana',
    'summary': 'Además, los expertos en terrorismo yihadista y el Parlamento '
               'Europeo han adver